In [1]:
## УДАЛИТЬ ЯЧЕЙКУ!!! ИНАЧЕ БУДЕТ БЕЗ GPU!!!
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
import keras
import numpy as np
import gymnasium as gym
import tensorflow as tf
import tqdm.notebook as tqdm
import matplotlib.pyplot as plt
import gymnasium.utils.save_video

2025-03-15 15:11:36.125469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-15 15:11:36.142640: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-15 15:11:36.142669: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-15 15:11:36.153981: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-15 15:11:37.012271: W tensorflow/compiler/tf

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
              tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

2025-03-15 15:11:38.465528: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-03-15 15:11:38.465562: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: gentoo
2025-03-15 15:11:38.465570: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: gentoo
2025-03-15 15:11:38.465710: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 550.107.2
2025-03-15 15:11:38.465733: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: 550.107.2
2025-03-15 15:11:38.465739: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:248] kernel version seems to match DSO: 550.107.2


In [5]:
batch_size = 64

In [6]:
train_env = gym.make_vec('LunarLander-v3', render_mode='rgb_array', num_envs=batch_size)
valid_env = gym.make_vec('LunarLander-v3', render_mode='rgb_array', num_envs=1)

In [7]:
def get_name(prefix: str | None = None, suffix: str | None = None, separator: str = '_') -> str | None:
    return prefix and prefix + separator + suffix or suffix or None

In [8]:
class ReplayBuffer:

    def __init__(self, max_size: int, observation_space: gym.spaces.Space, action_space: gym.spaces.Space, seed: int | None = None):
        """Stores the replay history with a maximum of `max_size` entries, removing old entries as needed.

        Parameters:
            max_size: maximal number of entries to keep
            observation_space: specification of the observation space
            action_space: specification of the action space
            seed: seed to initialize the internal random number generator for reproducibility"""

        self.current_observations = np.zeros((max_size, *observation_space.shape), dtype=observation_space.dtype)
        self.next_observations = np.zeros((max_size, *observation_space.shape), dtype=observation_space.dtype)
        self.actions = np.zeros((max_size, *action_space.shape), dtype=action_space.dtype)
        self.rewards = np.zeros((max_size,), dtype=np.float32)
        self.dones = np.zeros((max_size,), dtype=np.float32)
        
        self.max_size = max_size
        self.rng = np.random.default_rng(seed=seed)
        self.buffer_pointer = 0
        self.current_size = 0        
        
    def add(self, current_observations: np.ndarray, actions: np.ndarray, rewards: np.ndarray, next_observations: np.ndarray, dones: np.ndarray) -> None:
        """Add a new entry to the buffer.

        Parameters:
            current_observations: environment state observed at the current step
            actions: action taken by the model
            rewards: reward received after taking the action
            next_observations: environment state obversed after taking the action
            dones: whether the episode has ended or not"""
        batch_size = current_observations.shape[0]
        idxs = (np.arange(batch_size) + self.buffer_pointer) % self.max_size

        self.current_observations[idxs] = current_observations
        self.actions[idxs] = actions
        self.rewards[idxs] = rewards
        self.next_observations[idxs] = next_observations
        self.dones[idxs] = dones

        self.buffer_pointer = (idxs[-1] + 1) % self.max_size
        self.current_size = min(self.max_size, self.current_size + batch_size)
        
    def sample(self, n_samples: int, replace: bool = True) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """Randomly samples `n_samples` from the buffer.

        Parameters:
            n_samples: number of samples to select
            replace: sample with or without replacement

        Returns:
            current observations, actions, rewards, next observations, done"""
        return self[self.rng.choice(self.current_size, size=n_samples, replace=replace)]

    def clear(self) -> None:
        """Clears the buffer"""
        self.buffer_pointer = 0
        self.current_size = 0

    def __getitem__(self, index: int | np.ndarray) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """Gets a sample at `index`

        Parameters:
            index: index of the sample to get

        Returns:
            current observation, action, reward, next observation, done"""
        return (
            self.current_observations[index],
            self.actions[index],
            self.rewards[index],
            self.next_observations[index],
            self.dones[index]
        )
        
    def __len__(self) -> int:
        """Returns the number of entries in the buffer"""
        return self.current_size

In [9]:
def get_model(
    observation_space: gym.spaces.Space,
    action_space: gym.spaces.Space,
    features: int,
    blocks: int, 
    activation: str | keras.layers.Activation | None = 'silu',
    dropout_rate: float = 0.,
    multiply_freq: int = 1,
    name: str | None = None
) -> keras.Model:
    """Creates a simple MLP model

    Arguments:
        observation_space: model input vector size
        features: initial model embedding size
        action_space: model output vector size
        block: number of perceptron layers
        activation: intermediate model activation
        dropout_rate: nuff said
        multifly_freq: doubles embedding size every `multiply_freq` blocks
        name: model name

    Returns:
        A MLP model
    """
    inputs = x = keras.layers.Input(observation_space.shape, name=get_name(name, "inputs"), dtype=observation_space.dtype)

    for i in range(blocks):
        x = keras.layers.Dense(features, activation=activation, name=get_name(name, f"dense_{i}"))(x)
        
        if dropout_rate > 0.:
            x = keras.layers.Dropout(dropout_rate, name=get_name(name, f"dropout_{i}"))(x)

        if multiply_freq > 0 and (i + 1) % multiply_freq == 0:
            features *= 2
    
    out = keras.layers.Dense(action_space.n, name=get_name(name, "output"))(x)
    
    return keras.Model(inputs=inputs, outputs=out, name=name)

In [10]:
class QLoss(keras.layers.Layer):

    def __init__(self, gamma: float, **kwargs):
        super().__init__(**kwargs)
        
        self.gamma = gamma

    def call(self, q_current: keras.KerasTensor, q_next: keras.KerasTensor, rewards: keras.KerasTensor, actions: keras.KerasTensor, dones: keras.KerasTensor) -> keras.KerasTensor:
        q_ref = rewards + self.gamma * (1 - dones) * keras.ops.max(q_next, axis=-1)

        # return keras.ops.mean(
        #     keras.ops.square(keras.ops.sum(q_current * actions, axis=-1) - q_ref)
        # )
        return keras.ops.square(keras.ops.sum(q_current * actions, axis=-1) - q_ref)

    def get_config(self) -> dict:
        config = super().get_config()
        config['gamma'] = self.gamma
        return config

In [11]:
def get_combined_model(
    observation_space: gym.spaces.Space,
    action_space: gym.spaces.Space,    
    features: int,
    blocks: int, 
    activation: str | keras.layers.Activation | None = 'silu',
    dropout_rate: float = 0.,
    multiply_freq: int = 1,
    name: str | None = None,
    gamma: float = 0.99
) -> keras.Model:
    """Creates a combined model for Q-learning training

    Arguments:
        input_features: model input vector size
        features: initial model embedding size
        out_features: model output vector size
        block: number of perceptron layers
        activation: intermediate model activation
        dropout_rate: nuff said
        multifly_freq: doubles embedding size every `multiply_freq` blocks
        name: model name,
        gamma: rewards discount
    Returns:
        A q-model, a target q-model, a combined model
    """
    model = get_model(
        observation_space=observation_space,
        action_space=action_space,
        features=features,
        blocks=blocks,
        activation=activation,
        dropout_rate=dropout_rate,
        multiply_freq=multiply_freq,
        name=get_name(name, "model"),
    )
    
    target_model = get_model(
        observation_space=observation_space,
        action_space=action_space,
        features=features,
        blocks=blocks,
        activation=activation,
        multiply_freq=multiply_freq,
        name=get_name(name, "target_model"),
    )

    target_model.trainable = False
    target_model.set_weights(model.get_weights())

    current_observation = keras.layers.Input(observation_space.shape, dtype=observation_space.dtype, name=get_name(name, "curr_observ"))
    next_observation = keras.layers.Input(observation_space.shape, dtype=observation_space.dtype, name=get_name(name, "next_observ"))
    current_action = keras.layers.Input((action_space.n,), dtype="float32", name=get_name(name, "curr_action"))
    rewards = keras.layers.Input((), dtype="float32", name=get_name(name, "rewards"))
    dones = keras.layers.Input((), dtype="float32", name=get_name(name, "dones"))

    model_res = model(current_observation)
    target_model_res = target_model(next_observation)

    loss = QLoss(gamma=gamma, name=get_name(name, "Q_loss"))(model_res, target_model_res, rewards, current_action, dones)

    return (
        model, 
        target_model, 
        keras.Model(
            inputs=[
                current_observation,
                current_action,
                rewards,
                next_observation,
                dones,
            ],
            outputs=loss,
            name=name
        ),
    )

In [29]:
class Sampler:
    
    def __init__(self, epsilon: float, seed: int | None = None, greedy: bool = False):
        """Selects a random action with probability `epsilon` otherwise selects the most probably action given by the model.

        Parameters:
            epsilon: the probability to select a random action
            seed: seed to initialize the internal random number generator for reproducibility"""
        self.epsilon = epsilon
        self.greedy = greedy
        self.rng = np.random.default_rng(seed=seed)

    
    def _sample(self, probabilities: np.ndarray) -> np.ndarray:
        cumsum = np.cumsum(probabilities, axis=-1)
        probs = self.rng.uniform(size=(cumsum.shape[0], 1))
        msk = cumsum > probs
        return np.argmax(msk, axis=-1)

        
    def __call__(self, probabilities: np.ndarray) -> np.ndarray:
        """Select an action given the `probabilities

        Parameters:
            probabilities: probabilities for each action

        Returns:
            index of the selected action"""
        batch, *_ = probabilities.shape
        probs = self.rng.uniform(size=batch)

        return np.where(
            probs < self.epsilon,
            self.rng.choice(probabilities.shape[1], size=batch),
            np.argmax(probabilities, axis=-1) if self.greedy else self._sample(probabilities),
        )

In [13]:
def play_game(
    model: keras.Model,
    buffer: ReplayBuffer,
    env: gym.Env,
    steps: int,
    sampler: Sampler,
    observations: np.ndarray | None = None,
    one_episode: bool = False
) -> np.ndarray:
    """Plays the environment `env` using model `model` for a total of `steps` steps.

    Arguments:
        model: model to use
        buffer: buffer to store results to
        env: environment to use
        steps: total number of steps to record
        sampler: sampler to use
        observations: observation to start from
        one_episode: exist as soon as one of the environments finishes

    Returns:
        the last observations
    """
    if observations is None:
        observations, _ = env.reset()

    for _ in range(steps // env.num_envs):
        res = model(observations, training=False).numpy()
        actions = sampler(res)
        new_observations, rewards, terminateds, truncated, _ = env.step(actions) # new_observations, rewards, terminateds, truncated, info

        dones = terminateds | truncated

        buffer.add(
            current_observations=observations,
            actions=actions,
            rewards=rewards,
            next_observations=new_observations,
            dones=dones,
        )
        
        if one_episode and np.any(dones):
            observations = None
            break
        
        observations = new_observations
    
    return observations

In [14]:
class QDataset(keras.utils.PyDataset):

    def __init__(
        self,
        steps_per_epoch: int,
        batch_size: int,
        buffer: ReplayBuffer,
        action_space: gym.spaces.Space,
    ):
        super().__init__()
        self.buffer = buffer
        self.batch_size = batch_size
        self.steps_per_epoch = steps_per_epoch

        self._eye = np.eye(action_space.n)
        self._answer = np.zeros(batch_size, dtype=np.float32)

    def __len__(self):
        return self.steps_per_epoch

    def __getitem__(self, idx: int) -> np.ndarray:
        a, b, c, d, e = self.buffer.sample(self.batch_size)
        b = self._eye[b]

        return (a, b, c, d, e), self._answer

In [15]:
class ModelUpdateCallback(keras.callbacks.Callback):

    def __init__(self, frequency: int, **kwargs):
        super().__init__(**kwargs)
        self.frequency = frequency
        
    def on_train_batch_end(self, batch: int, logs=None):
        if (batch + 1) % self.frequency == 0:
            self.model.get_layer(
                f'{self.model.name}_target_model'
            ).set_weights(
                self.model.get_layer(
                    f'{self.model.name}_model'
                ).get_weights()
            )

In [16]:
class ModelEvalCallback(keras.callbacks.Callback):

    def __init__(self, env: gym.Env, max_steps: int, **kwargs):
        super().__init__(**kwargs)
        self.env = env
        self.max_steps = max_steps

        self.buffer = ReplayBuffer(
            max_size=max_steps,
            observation_space=env.single_observation_space,
            action_space=env.single_observation_space,
        )

        self.sampler = Sampler(epsilon=0., greedy=True)

    def on_epoch_end(self, epoch: int, logs=None):
        if logs is None:
            return

        model = self.model.get_layer(
            f'{self.model.name}_model'
        )

        self.buffer.clear()

        play_game(
            model=model,
            buffer=self.buffer,
            env=self.env,
            steps=self.max_steps,
            sampler=self.sampler,
            one_episode=True,
        )

        rewards = self.buffer.rewards[:len(self.buffer)].reshape((-1, self.env.num_envs))
        logs['score'] = np.mean(np.sum(rewards, axis=0))

In [17]:
class SometimesPlayCallback(keras.callbacks.Callback):
    def __init__(
        self,
        env: gym.Env,
        buffer: ReplayBuffer,
        sampler: Sampler,
        frequency: int,
        steps_per_play: int,
        epsilon_decay: keras.optimizers.schedules.PolynomialDecay | None = None,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.env = env
        self.buffer = buffer
        self.sampler = sampler
        self.frequency = frequency
        self.steps_per_play = steps_per_play
        self.epsilon_decay = epsilon_decay

        self.observations = None

    def on_epoch_begin(self, epoch: int, logs=None):
        if self.epsilon_decay is not None:
            self.sampler.epsilon = self.epsilon_decay(epoch)
        if epoch == 0:
            model = self.model.get_layer(
                f'{self.model.name}_model'
            )

            self.buffer.clear()
            
            self.observations = play_game(
                model=model,
                buffer=self.buffer,
                env=self.env,
                steps=self.steps_per_play,
                sampler=self.sampler,
                observations=self.observations
            )

    def on_train_batch_start(self, batch_num: int, logs=None):
        if (batch_num + 1) % self.frequency:
            return
        
        model = self.model.get_layer(
            f'{self.model.name}_model'
        )

        self.observations = play_game(
            model=model,
            buffer=self.buffer,
            env=self.env,
            steps=self.steps_per_play,
            sampler=self.sampler,
            observations=self.observations
        )

In [19]:
model, target_model, combined_model = get_combined_model(
    observation_space=train_env.single_observation_space,
    action_space=train_env.single_action_space,
    features=64,
    blocks=4,
    activation='silu',
    dropout_rate=0.2,
    multiply_freq=1,
    name='lander',
    gamma=0.99
)

In [20]:
combined_model.summary()

Model: "lander"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ lander_curr_observ  │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lander_next_observ  │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lander_model        │ (None, 4)         │    175,556 │ lander_curr_obse… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lander_target_model │ (None, 4)         │    175,556 │ lander_next_obse… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lander_rewards      │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lander_curr_action  │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lander_dones        │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lander_Q_loss       │ (None)            │          0 │ lander_model[0][… │
│ (QLoss)             │                   │            │ lander_target_mo… │
│                     │                   │            │ lander_rewards[0… │
│                     │                   │            │ lander_curr_acti… │
│                     │                   │            │ lander_dones[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 351,112 (1.34 MB)

 Trainable params: 175,556 (685.77 KB)

 Non-trainable params: 175,556 (685.77 KB)

In [21]:
buffer_size = 2 ** 15     # 32_768
steps_per_play = 2 ** 11  # 2048
steps_per_epoch = 2 ** 10 # 1024

end_epsilon = 0.1
epochs = 128
decay_epochs = epochs // 2
update_frequency = 512
play_frequency = 128

In [30]:
sampler = Sampler(1, greedy=False)

In [31]:
epsilon_decay = keras.optimizers.schedules.PolynomialDecay(1., decay_epochs, end_learning_rate=end_epsilon)

In [32]:
train_buffer = ReplayBuffer(
    max_size=buffer_size,
    observation_space=train_env.single_observation_space,
    action_space=train_env.single_action_space
)

In [33]:
dataset = QDataset(
    steps_per_epoch=steps_per_epoch,
    batch_size=train_env.num_envs,
    buffer=train_buffer,
    action_space=train_env.single_action_space
)

In [34]:
callbacks = [
    ModelUpdateCallback(update_frequency),
    ModelEvalCallback(valid_env, 1000),
    SometimesPlayCallback(
        env=train_env,
        buffer=train_buffer,
        sampler=sampler,
        frequency=play_frequency,
        steps_per_play=steps_per_play,
        epsilon_decay=epsilon_decay,
    )
]

In [35]:
combined_model.compile(
    loss=(lambda y_true, y_pred: keras.ops.mean(y_pred)),
    optimizer=keras.optimizers.Adam(1e-4, clipnorm=5, weight_decay=2e-5),
)

In [36]:
_ = play_game(
    model=model,
    buffer=train_buffer,
    env=train_env,
    steps=steps_per_play,
    sampler=sampler,
)

In [37]:
combined_model.fit(dataset, callbacks=callbacks, epochs=epochs)

Epoch 1/128


/home/excorador/source/nn/venv/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['lander_curr_observ', 'lander_curr_action', 'lander_rewards', 'lander_next_observ', 'lander_dones']. Received: the structure of inputs=('*', '*', '*', '*', '*')
  warnings.warn(


1024/1024 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 1.9953 - score: -447.7466
Epoch 2/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 1.3995 - score: -353.7941
Epoch 3/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 1.3067 - score: -363.3596
Epoch 4/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - loss: 1.4505 - score: -91.0697
Epoch 5/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 12s 11ms/step - loss: 1.6598 - score: -140.9352
Epoch 6/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step - loss: 1.7304 - score: -74.9289
Epoch 7/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 16s 16ms/step - loss: 1.9158 - score: -118.2722
Epoch 8/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 14s 14ms/step - loss: 1.9586 - score: -139.3293
Epoch 9/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - loss: 2.1048 - score: -89.8066
Epoch 10/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - loss: 2.2878 - score: -63.7563
Epoch 11/128
1024/1024 ━━━━━━━━━━━━━━━━━━━━ 14s 14ms/step - loss: 2.4656 - score: -127.3888
Epoch 12/128
1024/1

KeyboardInterrupt: 